# Day 22
https://adventofcode.com/2015/day/22

In [1]:
import aocd
data = aocd.get_data(year=2015, day=22)

In [2]:
from dataclasses import dataclass

##### Part 1: A wizard fights the boss

In [3]:
@dataclass(frozen=True)
class Spell():
    cost: int
    damage: int
    heal: int
    shield: int
    poison: int
    recharge: int

@dataclass(frozen=True)
class Situation():
    player_hp: int
    boss_hp: int
    boss_damage: int
    mana: int
    mana_spent: int
    effect_shield: int
    effect_poison: int
    effect_recharge: int
    
    @property
    def game_over(self):
        return self.boss_hp <= 0 or self.player_hp <= 0
    
    @property
    def has_won(self):
        return self.boss_hp <= 0 and self.player_hp > 0
    
    @property
    def has_lost(self):
        return self.player_hp <= 0
    
    @property
    def armor(self):
        return 7 if self.effect_shield > 0 else 0
    
    @property
    def poison(self):
        return 3 if self.effect_poison > 0 else 0
    
    @property
    def recharge(self):
        return 101 if self.effect_recharge > 0 else 0
    
    @property
    def available_mana(self):
        return self.mana + self.recharge

def castable(situation, spell):
    return (
        situation.available_mana >= spell.cost
        and not (situation.effect_shield > 1 and spell.shield > 0)
        and not (situation.effect_poison > 1 and spell.poison > 0)
        and not (situation.effect_recharge > 1 and spell.recharge > 0)
    )

def turn(situation, damage_player=0, damage_boss=0, spend_mana=0, add_shield=0, add_poison=0, add_recharge=0):
    # start of turn effects resolve:
    boss_hp = situation.boss_hp - situation.poison
    damage_player = 0 if boss_hp <= 0 else damage_player
    
    # then resolve the boss attack or spell:
    return Situation(
        situation.player_hp - damage_player,
        boss_hp - damage_boss,
        situation.boss_damage,
        situation.mana + situation.recharge - spend_mana,
        situation.mana_spent + spend_mana,
        max(0, situation.effect_shield - 1) + add_shield,
        max(0, situation.effect_poison - 1) + add_poison,
        max(0, situation.effect_recharge - 1) + add_recharge
    )

def boss_turn(situation):
    damage_dealt = max(1, situation.boss_damage - situation.armor)
    return turn(situation, damage_player=damage_dealt)

def player_turn(situation, spell, difficulty):
    return turn(
        situation,
        damage_player=difficulty-spell.heal,
        damage_boss=spell.damage,
        spend_mana=spell.cost,
        add_shield=spell.shield,
        add_poison=spell.poison,
        add_recharge=spell.recharge
    )

spellbook = {
    'missile': Spell(cost=53, damage=4, heal=0, shield=0, poison=0, recharge=0),
    'drain': Spell(cost=73, damage=2, heal=2, shield=0, poison=0, recharge=0),
    'shield': Spell(cost=113, damage=0, heal=0, shield=6, poison=0, recharge=0),
    'poison': Spell(cost=173, damage=0, heal=0, shield=0, poison=6, recharge=0),
    'recharge': Spell(cost=229, damage=0, heal=0, shield=0, poison=0, recharge=5),
}

def available_moves(situation, difficulty):
    return set(player_turn(situation, spell, difficulty)
               for spell in spellbook.values() if castable(situation, spell))

In [4]:
def load_initial_situation(text):
    from_text = dict((a.lower(), int(b)) for a, b in (line.split(': ') for line in text.split('\n')))
    return Situation(
        player_hp=50,
        boss_hp=from_text.get('hit points', 0),
        boss_damage=from_text.get('damage', 0),
        mana=500,
        mana_spent=0,
        effect_shield=0,
        effect_poison=0,
        effect_recharge=0
    )

In [5]:
def find_cheapest_win(initial, difficulty=0):
    candidates = list(available_moves(initial, difficulty))
    visited = set(candidates)
    while candidates:
        candidates.sort(key=lambda sit: sit.mana_spent, reverse=True)
        candidate = candidates.pop()
        
        if candidate.has_won:
            return candidate.mana_spent
        
        if not candidate.has_lost:
            after_boss = boss_turn(candidate)
            
            if after_boss.has_won: # eg. due to poison damage
                return after_boss.mana_spent
            
            if not after_boss.has_lost:
                for move in available_moves(after_boss, difficulty):
                    if move not in visited:
                        visited.add(move)
                        candidates.append(move)

In [6]:
initial = load_initial_situation(data)
p1 = find_cheapest_win(initial)
print('Part 1: {}'.format(p1))

Part 1: 1824


##### Part 2: Hard mode, where the player takes damage each turn!

In [7]:
p2 = find_cheapest_win(initial, difficulty=1)
print('Part 2: {}'.format(p2))

Part 2: 1937
